In [24]:
# здесь только импорт
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import math
import urllib2
from bs4 import BeautifulSoup
import string
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader.data as web
import copy
import progressbar
import time
import warnings
from datetime import datetime
from multiprocessing import Process
from joblib import Parallel, delayed, wrap_non_picklable_objects
import multiprocessing
from tqdm import tqdm
from tqdm import tqdm_notebook
from joblib import parallel_backend
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import operator

In [25]:
warnings.filterwarnings('ignore')

tickers_only = ['NOW', 'AMD', 'SE', 'PLAN', 'ZS', 'PD', 'SMAR', 'DDOG' , 'AVLR', 'OKTA', 'FSLY', 'TTD', 'PS', 'DOCU', 'BYND', 'COUP', 'DXCM', 'EGHT', 'AYX', 'RNG', 'PAYC', 'SEDG', 'OLED', 'ALRM', 'QTWO']
data_boost = pd.DataFrame(tickers_only)

for j in range(len(tickers_only)):
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[j] + '/' + tickers_only[j] + '/revenue')
        rg_table = table[1]
        rg_table = rg_table.iloc[:, 1].values
        rg_table = filter(lambda v: v==v, rg_table)
        rg_table = [s.replace('$', '') for s in rg_table]
        rg_table = [s.replace(',', '') for s in rg_table]
        x = map(float, rg_table) 
        x_rev = np.flip(x)
        x_rev = x_rev[-9:]
        qoe = list()
        for i in range(5):
            qoe.append(round(100 * (x_rev[i+4] - x_rev[i]) / x_rev[i], 2))
        qoe_mean = np.mean(qoe)
    except:
        qoe_mean = None
    data_boost.loc[j, 'mean_year_revenue_growth'] = qoe_mean     


In [26]:
now = datetime.now()
end_date = str(now.date())

for j in range(len(tickers_only)):
    stock_price = web.get_data_yahoo(tickers_only[j], start="2020-06-20", end = end_date)
    stock_price = stock_price['Adj Close'][len(stock_price)-1]
    data_boost.loc[j, 'stock_price'] = stock_price

    table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[j] + '/' + tickers_only[j] + '/eps-earnings-per-share-diluted')
    rg_table = table[1]
    rg_table = rg_table.iloc[:, 1].values
    rg_table = filter(lambda v: v==v, rg_table)
    rg_table = [s.replace('$', '') for s in rg_table]
    rg_table = [s.replace(',', '') for s in rg_table]
    x = map(float, rg_table) 
    x_rev = np.flip(x)
    
      
    X = np.linspace(1, len(x_rev), num=len(x_rev))
    X = X.reshape(-1, 1)
    polynomial_features= PolynomialFeatures(degree=1)
    x_poly = polynomial_features.fit_transform(X)
    model = LinearRegression()
    model.fit(x_poly, x_rev)
    y_poly_pred = model.predict(x_poly)
    r2 = r2_score(x_rev,y_poly_pred)
    
    data_boost.loc[j, 'R_square_linear'] = r2

    i = 0
    eps_sum = 0
    while stock_price > eps_sum:
        X_new = np.linspace(len(x_rev), len(x_rev)+i, num=(i+1))
        X_new = X_new.reshape(-1, 1)
        eps_sum = sum(model.predict(polynomial_features.fit_transform(X_new)))
        i = i + 1
        if i > 500:
            break
    data_boost.loc[j, 'years_to_payback_linear'] = i / 4.  
    
    
    
    X = np.linspace(1, len(x_rev), num=len(x_rev))
    X = X.reshape(-1, 1)
    polynomial_features= PolynomialFeatures(degree=2)
    x_poly = polynomial_features.fit_transform(X)
    model = LinearRegression()
    model.fit(x_poly, x_rev)
    y_poly_pred = model.predict(x_poly)
    r2 = r2_score(x_rev,y_poly_pred)
    
    data_boost.loc[j, 'R_square_parabolic'] = r2

    i = 0
    eps_sum = 0
    while stock_price > eps_sum:
        X_new = np.linspace(len(x_rev), len(x_rev)+i, num=(i+1))
        X_new = X_new.reshape(-1, 1)
        eps_sum = sum(model.predict(polynomial_features.fit_transform(X_new)))
        i = i + 1
        if i > 500:
            break
    data_boost.loc[j, 'years_to_payback_parabolic'] = i / 4.  

In [28]:
data_boost.sort_values(by = 'years_to_payback_parabolic')

,0,mean_year_revenue_growth,stock_price,R_square_linear,years_to_payback_linear,R_square_parabolic,years_to_payback_parabolic
5,PD,40.000,29.110001,0.112303,14.00,0.430689,3.50
14,BYND,220.882,141.679993,0.145866,15.00,0.662335,4.00
3,PLAN,43.024,43.950001,0.000986,60.00,0.079824,4.75
2,SE,148.114,105.199997,0.070071,125.25,0.310164,6.25
6,SMAR,53.444,52.980000,0.061146,19.00,0.084273,7.25
7,DDOG,NaN,84.500000,0.049000,47.00,0.457333,7.25
8,AVLR,38.762,134.139999,0.110519,13.00,0.117654,7.25
13,DOCU,39.048,177.580002,0.021371,16.00,0.025549,7.25
0,NOW,32.700,399.970001,0.054340,60.00,0.292903,14.50
1,AMD,12.734,50.099998,0.063430,23.25,0.065458,15.75
